In [19]:
import numpy as np

In [24]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 60.9 MB/s eta 0:00:00


In [25]:
from gurobipy import Model, GRB

In [21]:
# Define the technological matrix (A)
A_matrix = np.array([
    [2.5, 2, 1.8, 1.2],  # Ingredient Prep time constraint
    [12, 10, 8, 5],      # Formulation time constraint
    [10, 8, 6, 4],       # Packaging time constraint
    [0, 0, 0, -1],       # D must be at least 5 (D >= 5 → -D ≥ -5)
    [0, 1, 0, -1]        # B - D ≥ 0 (B >= D)
])

In [22]:
# Define the RHS vector (b)
b_vector = np.array([65, 95, 80, -5, 0])

In [23]:
# Define the objective function coefficients (c)
c_vector = np.array([20, 18, 15, 8])

In [26]:
# Define the primal LP problem using Gurobi
model = Model("Primal_LP")

Restricted license - for non-production use only - expires 2026-11-23


In [27]:
# Define decision variables (A, B, C, D)
variables = model.addVars(4, lb=0, name=["A", "B", "C", "D"])

In [28]:
# Set objective function (maximize profit)
model.setObjective(sum(c_vector[i] * variables[i] for i in range(4)), GRB.MAXIMIZE)

In [29]:
# Add constraints
for i in range(len(A_matrix)):
    model.addConstr(sum(A_matrix[i, j] * variables[j] for j in range(4)) <= b_vector[i], f"Constraint_{i+1}")

In [30]:
# Optimize model
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 4 columns and 15 nonzeros
Model fingerprint: 0xb59cd3f3
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [8e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 4 rows, 4 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7100000e+02   7.205450e+01   0.000000e+00      0s
       5    1.7125000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.712500000e+02


In [31]:
# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    print("Decision Variables (A, B, C, D):", [variables[i].x for i in range(4)])
    print("Optimal Objective Value:", model.objVal)
else:
    print("No optimal solution found.")

Optimal solution found:
Decision Variables (A, B, C, D): [0.0, 0.0, 8.75, 5.0]
Optimal Objective Value: 171.25


In [34]:
# Markdown Cell - Formulation of the Dual Problem
'''
### Dual Formulation:
Let \( y1, y2, y3, y4, y5 \) be the dual variables corresponding to the constraints:

\[
\min \quad 65y1 + 95y2 + 80y3 -5y4
\]
Subject to:
\[
2.5y1 + 12y2 + 10y3 \geq 20
\]
\[
2y1 + 10y2 + 8y3 + y5 \geq 18
\]
\[
1.8y1 + 8y2 + 6y3 \geq 15
\]
\[
1.2y1 + 5y2 + 4y3 - y4 - y5 \geq 8
\]
\[
y1, y2, y3, y4, y5 \geq 0
\]
'''

'\n### Dual Formulation:\nLet \\( y1, y2, y3, y4, y5 \\) be the dual variables corresponding to the constraints:\n\n\\[\n\\min \\quad 65y1 + 95y2 + 80y3 -5y4\n\\]\nSubject to:\n\\[\n2.5y1 + 12y2 + 10y3 \\geq 20\n\\]\n\\[\n2y1 + 10y2 + 8y3 + y5 \\geq 18\n\\]\n\\[\n1.8y1 + 8y2 + 6y3 \\geq 15\n\\]\n\\[\n1.2y1 + 5y2 + 4y3 - y4 - y5 \\geq 8\n\\]\n\\[\ny1, y2, y3, y4, y5 \\geq 0\n\\]\n'

In [35]:
# Define the dual problem
model_dual = Model("Dual_LP")

# Define dual decision variables (y1, y2, y3, y4, y5)
dual_variables = model_dual.addVars(5, lb=0, name=["y1", "y2", "y3", "y4", "y5"])

# Set dual objective function (minimize)
model_dual.setObjective(sum(b_vector[i] * dual_variables[i] for i in range(5)), GRB.MINIMIZE)

# Add dual constraints (transpose of A matrix)
for j in range(len(c_vector)):
    model_dual.addConstr(sum(A_matrix[i, j] * dual_variables[i] for i in range(5)) >= c_vector[j], f"Dual_Constraint_{j+1}")

# Optimize dual model
model_dual.optimize()

# Print dual results
if model_dual.status == GRB.OPTIMAL:
    print("Optimal solution found for Dual LP:")
    print("Dual Variables (y1, y2, y3, y4, y5):", [dual_variables[i].x for i in range(5)])
    print("Optimal Dual Objective Value:", model_dual.objVal)
else:
    print("No optimal solution found for Dual LP.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 4 rows, 5 columns and 15 nonzeros
Model fingerprint: 0x05443339
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [5e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 2e+01]
Presolve time: 0.01s
Presolved: 4 rows, 5 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.0000000e+30   1.000000e+30   5.000000e+00      0s
       3    1.7125000e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.712500000e+02
Optimal solution found for Dual LP:
Dual Variables (y1, y2, y3, y4, y5): [0.0, 1.875, 0.0, 1.375, 0.0]
Optimal Dual Objective Value: 171.25


In [36]:
# Write the model to a .lp file
model.write("model.lp")

In [37]:
# Write the dual model to a .lp file
model_dual.write("dual_model.lp")